# Convert Genesis DFL to Wavefront to SRWLWfr

This example will show how a DFL file produced by Genesis can be converted to an opemPMD-wavefront by lume-genesis, and then converted to an SRW wavefront object.

# Get Genesis v2 DFL file and write Wavefront

In [1]:
# Get example DFL file
# !wget -N https://portal.nersc.gov/cfs/m669/wavefronts/hxr_cut9_5keVTaper4_5.out.dfl

In [2]:
from h5py import File
from genesis.writers import write_openpmd_wavefront_h5

In [3]:
DFL = 'hxr_cut9_5keVTaper4_5.out.dfl'

# Genesis parameters. These need to be known to populate the openPMD-wavefront metadata
PARAM={'ncar': 251, 'dgrid': 0.00015, 'xlamds': 1.5005e-10, 'zsep': 20, 'ntail':0, 'itdp':1}

In [4]:
from genesis.parsers import parse_genesis_dfl

In [5]:
%%time
D2 = parse_genesis_dfl(DFL, nx=PARAM['ncar'])

CPU times: user 98.5 ms, sys: 430 ms, total: 528 ms
Wall time: 527 ms


In [6]:
PARAM['xlamds']*PARAM['zsep']

3.001e-09

In [7]:
with File('wavefront.h5', 'w') as h5:
    write_openpmd_wavefront_h5(h5, dfl=D2, param=PARAM)

In [8]:
h5 = File('wavefront.h5', 'r')
dict(h5['data']['000000']['meshes']['electricField'].attrs)

{'axisLabels': array(['x', 'y', 'z'], dtype=object),
 'frequency': 1.9979504031989338e+18,
 'frequencyUnitDimension': array([ 0,  0, -1,  0,  0,  0,  0]),
 'frequencyUnitSI': 1.0,
 'geometry': 'cartesian',
 'gridGlobalOffset': array([-1.50000e-04, -1.50000e-04, -7.47249e-07]),
 'gridSpacing': array([1.200e-06, 1.200e-06, 3.001e-09]),
 'gridUnitDimension': array([[1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0]]),
 'gridUnitSI': array([1., 1., 1.]),
 'photonEnergy': 8262.858873469322,
 'photonEnergyUnitDimension': array([ 2,  1, -2,  0,  0,  0,  0]),
 'photonEnergyUnitSI': 1.6021766208e-19,
 'timeOffset': 0.0,
 'unitDimension': array([ 1,  1, -3, -1,  0,  0,  0])}

# Wavefront to SRW

In [12]:
from pmd_wavefront.interfaces.srw import srw_wfr_from_openpmd_wavefront

In [13]:
with File('wavefront.h5', 'r') as h5:
    arEx, arEy, kwargs,  wrf_attrs = srw_wfr_from_openpmd_wavefront(h5['data']['000000']['meshes'],  iz_step=None)

In [15]:
# kwargs and attributes for the SRWLWfr class
kwargs,  wrf_attrs

({'_typeE': 'f',
  '_eStart': 0,
  '_eFin': 4.975098472957582e-15,
  '_ne': 498,
  '_xStart': -0.00015,
  '_xFin': 0.00015,
  '_nx': 251,
  '_yStart': -0.00015,
  '_yFin': 0.00015,
  '_ny': 251,
  '_zStart': 0},
 {'avgPhotEn': 8262.858805393349, 'presFT': 1, 'unitElFld': 2})